In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 데이터마트용 계정 생성
# sqlDeveloper에서 진행
# create user dm_admin IDENTIFIED BY "123qwe!@#QWE";
# grant dwrole to dm_admin;
# grant unlimited tablespace to dm_admin;


In [3]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 


In [4]:
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    rc('font', family='NanumGothic')

In [5]:
def cal_std_day(befor_day):   
    x = dt.datetime.now() - dt.timedelta(befor_day)
    year = x.year
    month = x.month if x.month >= 10 else '0'+ str(x.month)
    day = x.day if x.day >= 10 else '0'+ str(x.day)  
    return str(year)+ '-' +str(month)+ '-' +str(day)

### JDBC

In [6]:
#데이터마트
JDBC_DW = {
    'url':'jdbc:oracle:thin:@decorona1_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA1'
    ,'props':{
        'user':'dw_jjy'
       ,'password':'123qwe!@#QWE'
    }   
}
JDBC_DM = {
    'url':'jdbc:oracle:thin:@decorona1_high?TNS_ADMIN=/home/big/study/db/Wallet_DECORONA1'
    ,'props':{
        'user':'dm_jjy'
       ,'password':'123qwe!@#QWE'
    }   
}

## Load DataMart

### 1. 지역별 단위면적(km)당 인구수와 코로나 확진자 수 데이터 저장

In [7]:
popu=spark.read.jdbc(url=JDBC_DW['url'], table='loc',properties=JDBC_DW['props'])
patients=spark.read.jdbc(url=JDBC_DW['url'], table='corona_patients',properties=JDBC_DW['props'])
popu.show(3)
patients.show(3)


+----+----+----------+------------+
| LOC|AREA|POPULATION|FACILITY_CNT|
+----+----+----------+------------+
|서울| 605|   9736027|        5224|
|부산| 770|   3396109|        1462|
|대구| 883|   2412642|        1015|
+----+----+----------+------------+
only showing top 3 rows



+------+----+---------+-------+-----------+--------+----------+
|cp_idx| LOC|DEATH_CNT|DEF_CNT|LOC_OCC_CNT|QUR_RATE|   STD_DAY|
+------+----+---------+-------+-----------+--------+----------+
|   627|부산|     2253|1216922|       6629|   36322|2022-08-05|
|   628|인천|     1416|1147162|       4752|   38908|2022-07-28|
|   629|강원|      902| 608719|       4589|   39566|2022-08-11|
+------+----+---------+-------+-----------+--------+----------+
only showing top 3 rows



In [42]:
pop_patients = popu.join(patients, on='loc') \
                            .select('loc', 
                                   ceil(col('population')/col('area')).alias('POPU_DENSITY'),
                                        'qur_rate',
                                        'std_day') \
                                    .orderBy(col('std_day'))
pop_patients.show()


+----+------------+--------+----------+
| loc|POPU_DENSITY|qur_rate|   std_day|
+----+------------+--------+----------+
|광주|        2920|   39153|2022-07-28|
|세종|         811|   40272|2022-07-28|
|경남|         321|   35192|2022-07-28|
|강원|          93|   36318|2022-07-28|
|강원|          93|   36318|2022-07-28|
|경기|        1366|   39145|2022-07-28|
|경남|         321|   35192|2022-07-28|
|경북|         141|   32018|2022-07-28|
|경기|        1366|   39145|2022-07-28|
|경북|         141|   32018|2022-07-28|
|광주|        2920|   39153|2022-07-28|
|대구|        2733|   34103|2022-07-28|
|대구|        2733|   34103|2022-07-28|
|대전|        2722|   37839|2022-07-28|
|대전|        2722|   37839|2022-07-28|
|부산|        4411|   35083|2022-07-28|
|부산|        4411|   35083|2022-07-28|
|서울|       16093|   41322|2022-07-28|
|서울|       16093|   41322|2022-07-28|
|세종|         811|   40272|2022-07-28|
+----+------------+--------+----------+
only showing top 20 rows



In [27]:
# pop_patients = popu.join(patients, on='loc')\
#                 .select('loc', ceil(col('population')/col('area')).alias('POPU_DENSITY'),'qur_rate','std_day'.orderBy(col('std_day')))

AttributeError: 'str' object has no attribute 'orderBy'

In [43]:
pop_patients.write.jdbc(url=JDBC_DM['url'],table='CO_POPU_DENSITY', mode='append', properties=JDBC_DM['props'])

### 2.  백신접종 완료자와 코로나 확진자 수 데이터 저장

In [40]:
vaccine=spark.read.jdbc(url=JDBC_DW['url'],table='CORONA_VACCINE', properties=JDBC_DW['props'])

In [30]:
vaccine.show(3)

+------+----+----+--------+----------+
|CV_IDX| LOC|V_TH|   V_CNT|   STD_DAY|
+------+----+----+--------+----------+
|    30|경기|  v2|11712435|2022-09-18|
|    31|경기|  v4| 1688068|2022-09-18|
|    32|경기|  v1|11828148|2022-09-18|
+------+----+----+--------+----------+
only showing top 3 rows



In [49]:
pd_vaccine=vaccine.to_pandas_on_spark()
pd_vaccine=pd_vaccine.pivot_table(index=['LOC','STD_DAY'], columns='V_TH', values='V_CNT')
pd_vaccine=pd_vaccine.reset_index()
tmp=pd_vaccine.to_spark()

vac_rate=tmp.join(popu, on='LOC')\
                    .join(patients, on ='LOC')\
                    .select('LOC',patients.STD_DAY
                            ,ceil(col('v1')/col('population') * 100).alias('V1TH_RATE')
                            ,ceil(col('v1')/col('population') * 100).alias('V2TH_RATE')
                            ,ceil(col('v1')/col('population') * 100).alias('V3TH_RATE')
                            ,ceil(col('v1')/col('population') * 100).alias('V4TH_RATE')
                            ,'QUR_RATE'

       )
vac_rate.show(100)

+----+----------+---------+---------+---------+---------+--------+
| LOC|   STD_DAY|V1TH_RATE|V2TH_RATE|V3TH_RATE|V4TH_RATE|QUR_RATE|
+----+----------+---------+---------+---------+---------+--------+
|강원|2022-08-11|       87|       87|       87|       87|   39566|
|강원|2022-08-18|       87|       87|       87|       87|   41366|
|강원|2022-08-05|       87|       87|       87|       87|   37970|
|강원|2022-08-12|       87|       87|       87|       87|   39846|
|강원|2022-09-14|       87|       87|       87|       87|   45979|
|강원|2022-08-24|       87|       87|       87|       87|   42792|
|강원|2022-08-30|       87|       87|       87|       87|   43856|
|강원|2022-08-27|       87|       87|       87|       87|   43269|
|강원|2022-09-05|       87|       87|       87|       87|   44783|
|강원|2022-09-02|       87|       87|       87|       87|   44410|
|강원|2022-09-11|       87|       87|       87|       87|   45472|
|강원|2022-07-30|       87|       87|       87|       87|   36668|
|강원|2022-09-09|    

In [51]:
vac_rate.write.jdbc(url=JDBC_DM['url'], table='CO_VACCINE', mode='append', properties=JDBC_DM['props'])

### 3. 다중이용시설과 코로나 확진자 수의 상관관계

In [12]:
fac=spark.read.jdbc(url=JDBC_DW['url'],table='CORONA_FACILITY', properties=JDBC_DW['props'])

### 4. 요일별 코로나 확진자 수를 구해보자

In [ ]:
#TRANSFORMATION 파일에서 했었음